In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
c = 0
Dataset =[]
base_url = 'https://www.remax.co.za'
for page in range(1,5,1):
  html = urlopen(f'https://www.remax.co.za/property/for-sale/botswana/south-east-district/gaborone/?pt=1&page={page}')
  bs = BeautifulSoup(html.read(), 'html.parser')
  for property_link in bs.find_all('a', {'class':'property-card-link'}):
    Url = property_link.get('href')
    data = {
        'Url': base_url + Url
    }
    Dataset.append(data)

Dataset_Links = pd.DataFrame(Dataset)
#Filtered = Dataset_Links.iloc[::2]
print(len(Dataset_Links))
Dataset_Links.head(5)

160


,Url
0,https://www.remax.co.za/property/for-sale/bots...
1,https://www.remax.co.za/property/for-sale/bots...
2,https://www.remax.co.za/property/for-sale/bots...
3,https://www.remax.co.za/property/for-sale/bots...
4,https://www.remax.co.za/property/for-sale/bots...


In [3]:
Dataset_Links.drop_duplicates(subset = ['Url'], keep = 'first', inplace = True)
List = Dataset_Links['Url'].to_list()
print(len(List))

80


In [4]:
ReMax_Dataset = []
c = 0
for url in List:
  html = urlopen(url)
  soup = BeautifulSoup(html.read(), 'html.parser')
  Links = soup.find_all('div',{'class':'details property-details'})#Property Details
  for link in Links: 
    title = soup.find('div',{'class':'header-info-lrg'}).find('h1').get_text(strip= True)
    Bedrooms = link.find('td',{'itemprop':'numberOfRooms'}).get_text(strip= True)
    Bathrooms = link.find('td',{'itemprop':'amenityFeature'}).get_text(strip= True)
    try:
      Garage = link.find_all('td', {'itemprop':'amenityFeature'})[1].get_text(strip= True)
    except:
      Garage = 'N/A'
    Feature = soup.find('div',{'class':'details property-features'})
    try:
      Size = link.find('td',{'itemprop':'floorSize'}).get_text(strip= True)
    except:
      Size = 'N/A'

    Price = soup.find('div',{'class':'price listing_price'}).get_text(strip= True)

    Dataset1 = {
        'Title': title,
        'Bedrooms': Bedrooms,
        'Bathrooms': Bathrooms,
        'Garage': Garage,
        'Size': Size,
        'Price': Price
    }
    ReMax_Dataset.append(Dataset1)
    c += 1
print(f'Scraped {c} Houses')
Dataset_Part3 = pd.DataFrame(ReMax_Dataset)
Dataset_Part3

Scraped 80 Houses


,Title,Bedrooms,Bathrooms,Garage,Size,Price
0,4 Bedroom House For Sale in Block 8,4,3,N/A,615m2,BWP 1 800 000
1,3 Bedroom House For Sale in Phakalane Golf Estate,3,3,2,N/A,BWP 6 200 000
2,4 Bedroom House For Sale in Block 8,4,3,2,967m2,BWP 3 200 000
3,4 Bedroom House For Sale in Gaborone West Phase 1,4,4,N/A,1167m2,BWP 2 890 000
4,3 Bedroom House For Sale in Phakalane,3,2,2,708m2,BWP 2 350 000
...,...,...,...,...,...,...
75,3 Bedroom House For Sale in Gaborone North,3,2,N/A,859m2,BWP 1 820 000
76,5 Bedroom House For Sale in Phakalane,5,3,2,617m2,BWP 4 000 000
77,9 Bedroom House For Sale in Gaborone,9,1,N/A,405m2,BWP 800 000
78,5 Bedroom House For Sale in Extension 11,5,6,5,2000m2,BWP 28 000 000


In [5]:
Dataset_Part3.to_csv('ReMax.csv', encoding='utf-8', index = True)